In [ ]:
!pip install --no-binary numpy numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... canceledERROR: Operation cancelled by user


In [ ]:
!pip install pandas bertopic datamapplot==0.3.0 langdetect kaleido pandas==2.2.2 "scipy>=1.7.0,<1.14.0" "toolz>=0.11,<1.0" "dask==2024.12.1" optuna scikit-optimize langdetect glasbey

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
import pandas as pd
import numpy as np
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/AI Landscape Large Files/Data/constructed/')

# Load the data


In [ ]:
pub_data = pd.read_csv('pub_topic_modeling/As_final_final_sample.csv', index_col = 0)
pub_data = pd.read_csv('pub_topic_modeling/As_final_final_sample.csv', index_col = 0)
pub_data = pub_data[['Abstract', 'DOI', 'Cited by', 'Publication Year']]
pub_data = pub_data.rename(columns = {'Abstract': 'abstract'})
pub_data

,abstract,DOI,Cited by,Publication Year
2635,"For employees, work involves taking breaks as ...",10.4018/JOEUC.329596,1,2023
1864,The term 'artificial intelligence' (AI) refers...,10.1109/ICONSTEM60960.2024.10568724,0,2024
4681,This study aims to explore how firms' innovati...,10.1504/IJTM.2020.112122,5,2020
2308,This research aims to investigate the acceptan...,10.4018/IJEBR.323796,0,2023
1704,Generative AI appears poised to transform whit...,10.1145/3630106.3658987,0,2024
...,...,...,...,...
2451,"Since the introduction of ChatGPT, the leading...",10.1016/j.jbusres.2024.114720,2,2024
2713,IBM has deployed its Predictive Analytics for ...,10.1287/INTE.2020.1064,1,2021
119,Natural resources have long been considered a ...,10.1016/j.resourpol.2024.104831,0,2024
4419,Purpose: This study aims to explore the role t...,10.1108/IJRDM-12-2022-0493,7,2023


In [ ]:
from langdetect import detect_langs, LangDetectException
import pandas as pd

def is_english(text, threshold=0.95):
    """
    Determines whether a text is predominantly English.

    Parameters:
      text (str): The text to analyse.
      threshold (float): The minimum total probability for English for the text to be considered English.

    Returns:
      bool: True if the cumulative probability of English is at least the threshold, otherwise False.
    """
    try:
        # Skip empty or missing abstracts
        if pd.isna(text) or text.strip() == '':
            return False
        # Obtain the list of detected languages with their probabilities
        lang_probs = detect_langs(text)
        # Calculate the total probability of the text being English
        en_prob = sum(lang.prob for lang in lang_probs if lang.lang == 'en')
        return en_prob >= threshold
    except LangDetectException:
        # If there is an error during language detection, assume it's not English
        return False

In [ ]:
from langdetect import detect, LangDetectException

papers_topics_old = pd.read_csv('wp_topic_modeling/papers_topics_wp_prelim.csv', index_col = 0)
papers_topics_old = papers_topics_old[papers_topics_old['Topic'] != 3] # remove low accuracy
print(len(papers_topics_old))
papers_topics_old = papers_topics_old[papers_topics_old['abstract'].apply(is_english)] # remove non-english
print(len(papers_topics_old))
papers_topics_old = papers_topics_old.drop_duplicates(subset = ['abstract']) # remove duplicates
papers_topics_old.to_csv('wp_topic_modeling/papers_topics_nocits.csv')
print(len(papers_topics_old))
papers_topics_old = papers_topics_old[['abstract', 'doi']]
papers_topics_old

1217
1201
1107


,abstract,doi
0,Do stronger intellectual property rights incen...,10.3386/w32547
1,Drawing insights from the field of innovation ...,10.3386/w32474
2,How will the emergence of ChatGPT and other fo...,10.3386/w32430
3,David Ricardo initially believed machinery wou...,10.3386/w32416
4,Timely and accurate measurement of AI use by f...,10.3386/w32319
...,...,...
1294,Can artificial intelligence (AI) assist human ...,10.2139/ssrn.4397280
1295,This paper looks at transparency and fintech t...,NaN
1296,The knowledge-based and data-driven economy re...,10.2139/ssrn.3110785
1297,Credit risk is probably the most regulated app...,10.2139/ssrn.4615412


In [ ]:
# merge with citations

citations = pd.read_csv('wp_topic_modeling/citations.csv', index_col = 0)
citations = citations.drop_duplicates(subset = ['doi'])
papers_topics_cits = papers_topics_old.merge(citations, on = 'doi', how = 'left')

citations_nodois = pd.read_csv('wp_topic_modeling/citations_abstracts.csv', index_col = 0)
citations_nodois = citations_nodois.rename(columns = {'abstract': 'abstract_short'})
citations_nodois = citations_nodois.drop_duplicates(subset = ['abstract_short'])

papers_topics_cits['abstract_short'] = papers_topics_cits['abstract'].str[:500]
papers_topics_cits = papers_topics_cits.merge(citations_nodois, on = 'abstract_short', how = 'left')

papers_topics_cits['citation_count'] = papers_topics_cits['citation_count_x'].fillna(papers_topics_cits['citation_count_y'])
papers_topics_cits = papers_topics_cits.drop(columns = ['citation_count_x', 'citation_count_y', 'abstract_short'])

print(len(papers_topics_cits))
papers_topics_cits = papers_topics_cits.dropna(subset = 'citation_count') # drop papers without citations
print(len(papers_topics_cits))
papers_topics_cits = papers_topics_cits[['abstract', 'doi', 'citation_count']]
wp_data = papers_topics_cits
wp_data = wp_data.rename(columns = {'citation_count': 'Cited by'})
wp_data

1107
1028


,abstract,doi,Cited by
0,Do stronger intellectual property rights incen...,10.3386/w32547,0.0
1,Drawing insights from the field of innovation ...,10.3386/w32474,32.0
2,How will the emergence of ChatGPT and other fo...,10.3386/w32430,19.0
3,David Ricardo initially believed machinery wou...,10.3386/w32416,8.0
4,Timely and accurate measurement of AI use by f...,10.3386/w32319,23.0
...,...,...,...
1102,Can artificial intelligence (AI) assist human ...,10.2139/ssrn.4397280,237.0
1103,This paper looks at transparency and fintech t...,NaN,1.0
1104,The knowledge-based and data-driven economy re...,10.2139/ssrn.3110785,42.0
1105,Credit risk is probably the most regulated app...,10.2139/ssrn.4615412,1.0


In [ ]:
wp_data.to_csv('wp_topic_modeling/wp_full.csv')

In [ ]:
full_dataset = pd.concat([pub_data, wp_data])
full_dataset

,abstract,DOI,Cited by,Publication Year,doi
2635,"For employees, work involves taking breaks as ...",10.4018/JOEUC.329596,1.0,2023.0,NaN
1864,The term 'artificial intelligence' (AI) refers...,10.1109/ICONSTEM60960.2024.10568724,0.0,2024.0,NaN
4681,This study aims to explore how firms' innovati...,10.1504/IJTM.2020.112122,5.0,2020.0,NaN
2308,This research aims to investigate the acceptan...,10.4018/IJEBR.323796,0.0,2023.0,NaN
1704,Generative AI appears poised to transform whit...,10.1145/3630106.3658987,0.0,2024.0,NaN
...,...,...,...,...,...
1102,Can artificial intelligence (AI) assist human ...,NaN,237.0,NaN,10.2139/ssrn.4397280
1103,This paper looks at transparency and fintech t...,NaN,1.0,NaN,NaN
1104,The knowledge-based and data-driven economy re...,NaN,42.0,NaN,10.2139/ssrn.3110785
1105,Credit risk is probably the most regulated app...,NaN,1.0,NaN,10.2139/ssrn.4615412


# Add paper details

In [ ]:
pub_data_full = pd.read_csv("/content/drive/MyDrive/AI Landscape Large Files/Data/original/published_papers/no_duplicates.csv", index_col = 0).reset_index()
pub_data_full = pub_data_full[['Abstract', 'Article Title', 'Authors or Inventors']]
pub_data_full = pub_data_full.rename(columns = {'Abstract': 'abstract', 'Authors or Inventors' : 'author', 'Article Title': 'title'})
pub_data_full

,abstract,title,author
0,The backdrop of this research is the high glob...,Research on safe-haven currencies under global...,Lu C.; Yu F.; Li J.; Li S.
1,The COVID-19 pandemic has highlighted the pern...,Emotions unveiled: detecting COVID-19 fake new...,Farhoudinia B.; Ozturkcan S.; Kasap N.
2,This paper explores the ground-breaking approa...,Quantum machine learning in spatial analysis: ...,Vaz E.
3,Forest restoration has become a worldwide phen...,Superstition and attitudes towards restoration...,Arthur L.; Vondolia G.K.; Dasmani I.
4,The complexity of emotions has thus far limite...,A theory of emotion based on a universal model,Huang Y.
...,...,...,...
39755,The demand prediction becoming an essential to...,STUDY OF DIFFERENT DATA SCIENCE METHODS FOR DE...,"Iurasov, A; Stanelyte, G"
39756,Systems Thinking theorist J. P. Monat has hypo...,The self-awareness of the forest,"Monat, JP"
39757,How much can we learn finance directly from te...,How Much Can Machines Learn Finance from Chine...,"Zhou, Y; Fan, JQ; Xue, LR"
39758,This paper explores the factors that affect em...,Relevant factors that affect service recovery ...,"Lin, WB"


In [ ]:
wp_data_full=pd.read_csv("/content/drive/MyDrive/AI Landscape Large Files/Data/original/working_papers/working_papers.csv", index_col=0).reset_index()
wp_data_full = wp_data_full[['abstract', 'title', 'author']]
wp_data_full

,abstract,title,author
0,Understanding the behavior of users online is ...,webmunk: a new tool for studying online behavi...,"karr, chris"
1,The United States criminal justice system is c...,race and ethnicity (mis)measurement in the u.s...,"mueller-smith, michael g"
2,Despite its promise to help low-wealth househo...,breaking boundaries: lower tail dependence can...,"escribano, álvaro"
3,"Over the past two decades, respondents to the ...",emotions and subjective crash beliefs,"shiller, robert j"
4,Do stronger intellectual property rights incen...,do stronger ipr incentivize female participati...,"higgins, matthew j"
...,...,...,...
8715,This paper identifies U.S. monetary and fiscal...,classification of monetary and fiscal dominanc...,"hinterlang, natascha, hollmayr, josef"
8716,The COVID-19 pandemic forced performing arts o...,managing supply and demand for the performing ...,"nattamai kannan, karthik babu"
8717,Energy market volatility affects macroeconomic...,artificial neural network models for forecasti...,"al-fattah, saud"
8718,The current study reviews various literatures ...,application of evolutionary computation techni...,"chattopadhyay, subhajit"


In [ ]:
full_dataset = full_dataset.merge(pub_data_full, on = 'abstract', how = 'left')
full_dataset = full_dataset.merge(wp_data_full, on = 'abstract', how = 'left')
full_dataset['title'] = full_dataset['title_y'].fillna(full_dataset['title_x'])
full_dataset['authors'] = full_dataset['author_y'].fillna(full_dataset['author_x'])
full_dataset = full_dataset.drop(columns = ['title_x', 'author_x', 'title_y', 'author_y'])
full_dataset = full_dataset.drop_duplicates(subset = ['abstract'])
full_dataset

,abstract,DOI,Cited by,Publication Year,doi,title,authors
0,"For employees, work involves taking breaks as ...",10.4018/JOEUC.329596,1.0,2023.0,NaN,The Shape of Workbreaks to Come: Reframing Cyb...,Oravec J.A.
1,The term 'artificial intelligence' (AI) refers...,10.1109/ICONSTEM60960.2024.10568724,0.0,2024.0,NaN,A Novel Approach of Data-Driven Strategic Deci...,Umamaheswari S.; Valarmathi A.; Dhinakaran D.P...
2,This study aims to explore how firms' innovati...,10.1504/IJTM.2020.112122,5.0,2020.0,NaN,How to become the chosen one in the artificial...,Li J.; Liu Z.; Zhou J.
3,This research aims to investigate the acceptan...,10.4018/IJEBR.323796,0.0,2023.0,NaN,A Study of Customer Acceptance of Artificial I...,Choi Y.
4,Generative AI appears poised to transform whit...,10.1145/3630106.3658987,0.0,2024.0,NaN,The Impact and Opportunities of Generative AI ...,Wolfe R.; Mitra T.
...,...,...,...,...,...,...,...
4656,Can artificial intelligence (AI) assist human ...,NaN,237.0,NaN,10.2139/ssrn.4397280,when and how artificial intelligence augments ...,"jia, nan, luo, xueming , fang, zheng, liao, ch..."
4657,This paper looks at transparency and fintech t...,NaN,1.0,NaN,NaN,demystifying consumer-facing fintech: accounta...,"paterson, jeannie marie, miller, tim, lyons, h..."
4658,The knowledge-based and data-driven economy re...,NaN,42.0,NaN,10.2139/ssrn.3110785,digital trade: is data treaty-ready?,"ciuriak, dan"
4659,Credit risk is probably the most regulated app...,NaN,1.0,NaN,10.2139/ssrn.4615412,credit risk and artificial intelligence: on th...,"bravo, cristián, calabrese, raffaella, lessman..."


In [ ]:
full_dataset.to_csv('pub_wp_full.csv')
full_dataset

,abstract,DOI,Cited by,Publication Year,doi,title,authors
0,"For employees, work involves taking breaks as ...",10.4018/JOEUC.329596,1.0,2023.0,NaN,The Shape of Workbreaks to Come: Reframing Cyb...,Oravec J.A.
1,The term 'artificial intelligence' (AI) refers...,10.1109/ICONSTEM60960.2024.10568724,0.0,2024.0,NaN,A Novel Approach of Data-Driven Strategic Deci...,Umamaheswari S.; Valarmathi A.; Dhinakaran D.P...
2,This study aims to explore how firms' innovati...,10.1504/IJTM.2020.112122,5.0,2020.0,NaN,How to become the chosen one in the artificial...,Li J.; Liu Z.; Zhou J.
3,This research aims to investigate the acceptan...,10.4018/IJEBR.323796,0.0,2023.0,NaN,A Study of Customer Acceptance of Artificial I...,Choi Y.
4,Generative AI appears poised to transform whit...,10.1145/3630106.3658987,0.0,2024.0,NaN,The Impact and Opportunities of Generative AI ...,Wolfe R.; Mitra T.
...,...,...,...,...,...,...,...
4656,Can artificial intelligence (AI) assist human ...,NaN,237.0,NaN,10.2139/ssrn.4397280,when and how artificial intelligence augments ...,"jia, nan, luo, xueming , fang, zheng, liao, ch..."
4657,This paper looks at transparency and fintech t...,NaN,1.0,NaN,NaN,demystifying consumer-facing fintech: accounta...,"paterson, jeannie marie, miller, tim, lyons, h..."
4658,The knowledge-based and data-driven economy re...,NaN,42.0,NaN,10.2139/ssrn.3110785,digital trade: is data treaty-ready?,"ciuriak, dan"
4659,Credit risk is probably the most regulated app...,NaN,1.0,NaN,10.2139/ssrn.4615412,credit risk and artificial intelligence: on th...,"bravo, cristián, calabrese, raffaella, lessman..."


# Topic model

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
documents = full_dataset['abstract']
documents = documents.reset_index(drop=True)
documents = documents.tolist()
documents = [str(doc) for doc in documents if isinstance(doc, str)]

In [ ]:
# Function to get coherence score

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from gensim import corpora

def calculate_coherence_score(topic_model, docs):
    # Preprocess documents
    cleaned_docs = topic_model._preprocess_text(docs)

    # Extract vectorizer and tokenizer from BERTopic
    vectorizer = topic_model.vectorizer_model
    tokenizer = vectorizer.build_tokenizer()

    # Extract features for Topic Coherence evaluation
    words = vectorizer.get_feature_names_out()
    # depending on the version and if you get an error use commented out code below:
    # words = vectorizer.get_feature_names()
    tokens = [tokenizer(doc) for doc in cleaned_docs]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]
    # Create topic words
    # topic_words = [[dictionary.token2id[w] for w in words if w in dictionary.token2id] for _ in range(topic_model.nr_topics)]
    topic_words = [[words for words, _ in topic_model.get_topic(topic)]
               for topic in range(len(set(topics))-1)]

    # this creates a list of the token ids (in the format of integers) of the words in words that are also present in the
    # dictionary created from the preprocessed text. The topic_words list contains list of token ids for each
    # topic.

    coherence_model = CoherenceModel(topics=topic_words,
                                    texts=tokens,
                                    corpus=corpus,
                                    dictionary=dictionary,
                                    coherence='c_v')
    coherence = coherence_model.get_coherence()

    return coherence

In [ ]:
# function for getting topic model
def create_topic_model(documents, min_cluster_size):
  hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, metric='euclidean',
                          cluster_selection_method='eom', prediction_data=True)

  # Initialise the BERTopic model
  topic_model = BERTopic(
      embedding_model=None,
      umap_model=UMAP(n_neighbors=15, n_components=5, metric='cosine', random_state=42),
      hdbscan_model=hdbscan_model,
      vectorizer_model=CountVectorizer(stop_words="english")
  )

  # get embeddings
  embedding_model = SentenceTransformer("all-roberta-large-v1")
  embeddings = embedding_model.encode(documents, show_progress_bar=True)
  embeddings = embeddings.astype(np.float64)

  # Fit the model on your data
  topics, probs = topic_model.fit_transform(documents, embeddings)

  # return topics, probs, topic model
  return topics, probs, topic_model

# # run
# topics, probs, topic_model = create_topic_model(documents, 25)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/145 [00:00<?, ?it/s]

In [ ]:
import numpy
import matplotlib.pyplot as plt

# Define parameter range for min_cluster_size
min_cluster_sizes = range(10, 80, 5)
coherences = []
num_topics = []
min_clusters = []
uncategorized_topics = []

for min_cluster_size in min_cluster_sizes:
    # Update the HDBSCAN model with the new min_cluster_size
    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, metric='euclidean',
                            cluster_selection_method='eom', prediction_data=True)

    # Initialise the BERTopic model
    topic_model = BERTopic(
        embedding_model=None,
        umap_model=UMAP(n_neighbors=15, n_components=5, metric='cosine', random_state=42),
        hdbscan_model=hdbscan_model,
        vectorizer_model=CountVectorizer(stop_words="english")
    )
    topics, probs = topic_model.fit_transform(documents, embeddings)

    # Get coherence
    coherence = calculate_coherence_score(topic_model, documents)

    # Get the number of uncategorised topics
    uncategorized = topics.count(-1)

    # Append to list
    coherences.append(coherence)
    num_topics.append(len(set(topics)) - (1 if -1 in topics else 0))  # Number of topics
    min_clusters.append(min_cluster_size)
    uncategorized_topics.append(uncategorized)
    print(f"Min cluster size: {min_cluster_size}, Coherence: {coherence}, Number of uncategorized: {uncategorized}")

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(min_clusters, coherences, marker='o')
plt.xlabel("Min topic size")
plt.ylabel("Coherence")
plt.title("Coherence vs Min topic size")
plt.show()

In [ ]:
topic_df = topic_model.get_topic_info()
topic_df.to_csv('topics_full.csv')
topic_df

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1528,-1_ai_data_study_artificial,"[ai, data, study, artificial, intelligence, re...",[Research background:The climate crisis is one...
1,0,345,0_labor_employment_automation_workers,"[labor, employment, automation, workers, labou...",[Sustainable Development Goal 8 proposes the p...
2,1,304,1_financial_banking_banks_credit,"[financial, banking, banks, credit, services, ...",[The contemporary financial landscape is witne...
3,2,217,2_hr_hrm_management_ai,"[hr, hrm, management, ai, recruitment, human, ...",[Artificial intelligence (AI) has emerged as a...
4,3,197,3_ai_business_innovation_adoption,"[ai, business, innovation, adoption, value, or...",[It has been claimed that Artificial Intellige...
5,4,197,4_manufacturing_40_production_industry,"[manufacturing, 40, production, industry, tech...","[In this paper, we propose a new smart manufac..."
6,5,191,5_chatgpt_generative_llms_students,"[chatgpt, generative, llms, students, language...","[Generative artificial intelligence (AI), such..."
7,6,185,6_supply_chain_resilience_logistics,"[supply, chain, resilience, logistics, study, ...",[The escalating complexity of dynamic supply c...
8,7,176,7_innovation_firms_gpt_patent,"[innovation, firms, gpt, patent, knowledge, pa...","[This research report (in Korean), submitted a..."
9,8,157,8_ai_public_governance_systems,"[ai, public, governance, systems, regulation, ...",[Artificial Intelligence systems (“AI”) are of...


In [ ]:
import pickle

# Save the model
with open('full_paper_model.pkl', 'wb') as f:
    pickle.dump(topic_model, f)

print("Model saved successfully!")

# load model
with open('full_paper_model.pkl', 'rb') as f:
    topic_model = pickle.load(f)

Model saved successfully!


In [ ]:
# load model
with open('full_paper_model.pkl', 'rb') as f:
    topic_model = pickle.load(f)

In [ ]:
custom_topic_labels = {
    -1: "",
    0: "Labour & Automation",
    1: "Financial Services & Banking",
    2: "Human Resource Management",
    3: "Business Innovation & Adoption",
    4: "Manufacturing & Industry 4.0",
    5: "Generative AI",
    6: "Supply Chain & Logistics",
    7: "Innovation",
    8: "AI Governance & Regulation",
    9: "Service Robots",
    10: "Healthcare",
    11: "Energy and Sustainability",
    12: "Accounting and Auditing",
    13: "Chatbots and Customer Service",
    14: "Corporate Disclosure",
    15: "ESG",
    16: "Decision-Making & Algorithms",
    17: "Consumer Retail Experience",
    18: "Marketing and CRM",
    19: "Construction & Projuect Management",
    20: "Employee Awareness",
    21: "Algrotihms and Pricing",
    22: "Labour Regulation",
    23: "Legal Profession & Law"
}

topic_model.set_topic_labels(custom_topic_labels)

SyntaxError: expression expected after dictionary key and ':' (<ipython-input-28-2e101c15a99d>, line 7)

# Outlier classification

# Visualize

In [ ]:
import matplotlib.font_manager
from tempfile import NamedTemporaryFile
from fontTools import ttLib
import re
import requests

def get_google_font(fontname):
    api_fontname = fontname.replace(' ', '+')
    api_response = resp = requests.get(f"https://fonts.googleapis.com/css?family={api_fontname}:black,bold,regular,light")
    font_urls = re.findall(r'(https?://[^\)]+)', str(api_response.content))
    for font_url in font_urls:
        font_data = requests.get(font_url)
        f = NamedTemporaryFile(delete=False, suffix='.ttf')
        f.write(font_data.content)
        f.close()
        font = ttLib.TTFont(f.name)
        font_family_name = font['name'].getDebugName(1)
        matplotlib.font_manager.fontManager.addfont(f.name)
        print(f"Added new font as {font_family_name}")

get_google_font("Roboto")

In [ ]:
docs_df = pd.DataFrame({'abstract': documents})
papers_topics = pd.DataFrame({'abstract': documents, 'Topic': topics, 'Probability': probs})
papers_topics = papers_topics.merge(full_dataset[['abstract', 'Cited by', 'title', 'authors']], on = 'abstract', how = 'left')
docs_cits = docs_df.merge(papers_topics[['abstract', 'Cited by', 'title', 'authors']], on = 'abstract', how = 'right')[['abstract', 'Cited by', 'title', 'authors']]
docs_cits = docs_cits.dropna(subset = 'Cited by')
docs_cits = docs_cits.drop_duplicates(subset = ['abstract'])
docs_cits

,abstract,Cited by,title,authors
0,"For employees, work involves taking breaks as ...",1.0,The Shape of Workbreaks to Come: Reframing Cyb...,Oravec J.A.
1,The term 'artificial intelligence' (AI) refers...,0.0,A Novel Approach of Data-Driven Strategic Deci...,Umamaheswari S.; Valarmathi A.; Dhinakaran D.P...
2,This study aims to explore how firms' innovati...,5.0,How to become the chosen one in the artificial...,Li J.; Liu Z.; Zhou J.
3,This research aims to investigate the acceptan...,0.0,A Study of Customer Acceptance of Artificial I...,Choi Y.
4,Generative AI appears poised to transform whit...,0.0,The Impact and Opportunities of Generative AI ...,Wolfe R.; Mitra T.
...,...,...,...,...
4622,Can artificial intelligence (AI) assist human ...,237.0,when and how artificial intelligence augments ...,"jia, nan, luo, xueming , fang, zheng, liao, ch..."
4623,This paper looks at transparency and fintech t...,1.0,demystifying consumer-facing fintech: accounta...,"paterson, jeannie marie, miller, tim, lyons, h..."
4624,The knowledge-based and data-driven economy re...,42.0,digital trade: is data treaty-ready?,"ciuriak, dan"
4625,Credit risk is probably the most regulated app...,1.0,credit risk and artificial intelligence: on th...,"bravo, cristián, calabrese, raffaella, lessman..."


In [ ]:
import umap
import datamapplot
import numpy as np
from sentence_transformers import SentenceTransformer
import numpy as np
import plotly.io as pio
import datamapplot
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
import glasbey

In [ ]:
# -----------------------------------------------------------------------------
# 1. Re‐encode abstracts for the filtered dataset (docs_cits)
# -----------------------------------------------------------------------------

# Get embeddings
embedding_model = SentenceTransformer("all-roberta-large-v1")
embeddings = embedding_model.encode(documents, show_progress_bar=True)
embeddings = embeddings.astype(np.float64)

# Reduce the embeddings to 2D for plotting.
umap_reducer = umap.UMAP(n_components=2, random_state=42)
data_map = umap_reducer.fit_transform(embeddings)

In [ ]:
# -----------------------------------------------------------------------------
# 2. Compute the marker sizes by scaling the citation counts.
# -----------------------------------------------------------------------------
# Convert citation counts to float and scale so the maximum marker size is 50.
citations = docs_cits["Cited by"].astype(float).values
scaled_citations = np.asarray(10 + 40 * np.log10(1+ citations) / np.log10(1+ citations.max()))

# -----------------------------------------------------------------------------
# 3. Create custom hover text.
# -----------------------------------------------------------------------------
# Each entry will show the paper’s title, authors and citation count.
hover_text = [
    f"Title: {title}Authors: {authors} Citations: {cit}"
    for title, authors, cit in zip(docs_cits['title'], docs_cits['authors'], docs_cits["Cited by"])
]

# -----------------------------------------------------------------------------
# 4. Prepare a label layer from your BERTopic output.
# -----------------------------------------------------------------------------
# Merge in the topic numbers (if not already present) from papers_topics.
docs_cits = docs_cits.merge(papers_topics[['abstract', 'Topic']], on='abstract', how='left')
# Map each topic number to a custom topic label.
label_layer = [
    custom_topic_labels[t] if t in custom_topic_labels else str(t)
    for t in docs_cits['Topic']
]

# -----------------------------------------------------------------------------
# 5. Assign custom colours using Glasbey palette.
# -----------------------------------------------------------------------------
# First create a mapping from topic numbers to labels
topic_to_label = {}
for t in set(docs_cits['Topic']):
    if t in custom_topic_labels:
        topic_to_label[t] = custom_topic_labels[t]
    else:
        topic_to_label[t] = str(t)

# Now create your label layer using these labels
label_layer = [topic_to_label[t] for t in docs_cits['Topic']]

# Get a list of unique topics excluding -1
positive_topics = [t for t in set(docs_cits['Topic']) if t >= 0]
num_positive_topics = len(positive_topics)

# Import glasbey and generate a palette
import glasbey

# Generate a Glasbey palette with good perceptual distinction
colors = glasbey.create_palette(num_positive_topics, chroma_bounds=(30, 80), lightness_bounds=(25, 75))

# Create your custom colors dictionary mapping LABELS to colors
label_color_map = {}

# Assign Glasbey colors to positive topics
for i, t in enumerate(positive_topics):
    # Get color from Glasbey palette
    color = colors[i]
    # Convert to RGBA tuple (adding alpha=1.0)
    rgba_tuple = tuple(color) + (1.0,)
    label_color_map[topic_to_label[t]] = rgba_tuple

# Assign grey to topic -1 if it exists
if -1 in set(docs_cits['Topic']):
    label_color_map[topic_to_label[-1]] = (0.7, 0.7, 0.7, 1.0)

# Handle any empty string labels that might exist
if '' in label_layer:
    label_color_map[''] = (0.7, 0.7, 0.7, 1.0)

# -----------------------------------------------------------------------------
# 6. Create the interactive DataMapPlot.
# -----------------------------------------------------------------------------
plot = datamapplot.create_interactive_plot(
    data_map,
    label_layer,
    hover_text = hover_text,
    font_family = "Roboto",
    min_fontsize=14,
    max_fontsize=14,
    initial_zoom_fraction=0.8,
    marker_size_array = scaled_citations,
    title = "The Economics of AI Landscape",
    sub_title = "Data map of Economics of AI papers; scaled by citations",
    enable_search = True,
    label_color_map=label_color_map,  # Use the color map with Glasbey colors
    darkmode = False,
)

# Optionally, save the plot to an HTML file for sharing.
plot.save("AI_Landscape_Interactive.html")

<ipython-input-76-5060b1936707>:61: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('viridis', num_positive_topics)


In [ ]:
# ## Static Plot

# citations = docs_cits['Cited by'].to_list()
# scaled_citations = [2*(c / max(citations)) * 100 for c in citations]
# fig = topic_model.visualize_document_datamap(documents, custom_labels = True, embeddings=embeddings, point_size=scaled_citations, title="The Economics of AI Landscape", sub_title="Data map of Economics of AI papers from Scopus, WoS, NBER and SSRN; scaled by citations")
# fig.show()
# # fig.savefig("full_landscape.pdf", dpi=600, bbox_inches='tight')

TypeError: BERTopic.visualize_document_datamap() got an unexpected keyword argument 'point_size'